In [6]:
import os
import numpy as np
from utils import imshow
import nibabel as nib

img = nib.load('../data/19981029103253/19981029103253_CHEST_ABD_PELVIS_102.nii.gz')

# img.shape

dat = img.get_fdata()
print(type(dat)) #np.array
print(dat.shape)
# imshow(dat)

imshow(dat[128, 128, 15])
imshow(dat[..., 52])

imshow(dat[..., 300])


# imshow(dat[..., 0], title='T2')
# imshow(dat[..., 1], title='FLAIR')
# imshow(dat[..., 2], title='T1 precontrast')
# imshow(dat[..., 3], title='T1 postcontrast')

<class 'numpy.ndarray'>
(512, 512, 265)


TypeError: Invalid dimensions for image data